# Je commence par importer les librairies dont j'aurais besoin pour la suite du travail

In [1]:
import pandas as pd
import numpy as np
import re

# Importation du fichier

In [2]:
df = pd.read_csv('GSAF5.csv', sep=',',engine='python', encoding='latin1')
#df

In [3]:
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


# Lecture des infos

Je commence par lire les infos macro du fichier

In [4]:
df.shape

(5992, 24)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5992 entries, 0 to 5991
Data columns (total 24 columns):
Case Number               5992 non-null object
Date                      5992 non-null object
Year                      5992 non-null int64
Type                      5992 non-null object
Country                   5949 non-null object
Area                      5590 non-null object
Location                  5496 non-null object
Activity                  5465 non-null object
Name                      5792 non-null object
Sex                       5425 non-null object
Age                       3311 non-null object
Injury                    5965 non-null object
Fatal (Y/N)               5973 non-null object
Time                      2779 non-null object
Species                   3058 non-null object
Investigator or Source    5977 non-null object
pdf                       5992 non-null object
href formula              5991 non-null object
href                      5989 non-null object
C

# Inspection des colonnes

In [6]:
# Column Consistency
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [7]:
#df.rename(columns={'Sex ':'Sex','Species ':'Species'})

In [8]:
df.columns.str.upper()

Index(['CASE NUMBER', 'DATE', 'YEAR', 'TYPE', 'COUNTRY', 'AREA', 'LOCATION',
       'ACTIVITY', 'NAME', 'SEX ', 'AGE', 'INJURY', 'FATAL (Y/N)', 'TIME',
       'SPECIES ', 'INVESTIGATOR OR SOURCE', 'PDF', 'HREF FORMULA', 'HREF',
       'CASE NUMBER.1', 'CASE NUMBER.2', 'ORIGINAL ORDER', 'UNNAMED: 22',
       'UNNAMED: 23'],
      dtype='object')

In [9]:
df.columns = df.columns.str.upper()
df.columns

Index(['CASE NUMBER', 'DATE', 'YEAR', 'TYPE', 'COUNTRY', 'AREA', 'LOCATION',
       'ACTIVITY', 'NAME', 'SEX ', 'AGE', 'INJURY', 'FATAL (Y/N)', 'TIME',
       'SPECIES ', 'INVESTIGATOR OR SOURCE', 'PDF', 'HREF FORMULA', 'HREF',
       'CASE NUMBER.1', 'CASE NUMBER.2', 'ORIGINAL ORDER', 'UNNAMED: 22',
       'UNNAMED: 23'],
      dtype='object')

# point de vigilance
En voulant travailler sur la colonne "sex", ça indiquait "undefined" --> il y a un problème de syntaxe. <br/>
il faut donc renommer la colonne pour supprimer l'espace en trop.<br>
pareil pour la colonne "species"
J'en profite pour mettre les noms de colonne en majuscule

In [14]:
df = df.rename(columns={'SEX ':'SEX','SPECIES ':'SPECIES'})

df.columns

Index(['CASE NUMBER', 'DATE', 'YEAR', 'TYPE', 'COUNTRY', 'AREA', 'LOCATION',
       'ACTIVITY', 'NAME', 'SEX', 'AGE', 'INJURY', 'FATAL (Y/N)', 'TIME',
       'SPECIES', 'INVESTIGATOR OR SOURCE', 'PDF', 'HREF FORMULA', 'HREF',
       'CASE NUMBER.1', 'CASE NUMBER.2', 'ORIGINAL ORDER', 'UNNAMED: 22',
       'UNNAMED: 23'],
      dtype='object')

In [16]:
#df = df.rename(columns={'CASE NUMBER ':'CASE_NUMBER','FATAL (Y/N) ':'FATAL_(Y/N)','INVESTIGATOR OR SOURCE ':'INVESTIGATOR_OR_SOURCE','HREF FORMULA ':'HREF_FORMULA','CASE NUMBER.1 ':'CASE_NUMBER_1','CASE NUMBER.2 ':'CASE_NUMBER_2','ORIGINAL ORDER ':'ORIGINAL_ORDER','UNNAMED: 22 ':'UNNAMED_22','UNNAMED: 23 ':'UNNAMED_23'})
#df.columns

In [17]:
#pas forcément necessaire et lisible mais me permet d'identifier sur quelles colonnes travailller
for col in df.columns:
    print(df[col].value_counts())

1962.06.11.b      2
1907.10.16.R      2
1913.08.27.R      2
1966.12.26        2
2005.04.06        2
2009.12.18        2
1980.07.00        2
2014.08.02        2
1983.06.15        2
2013.10.05        2
1923.00.00.a      2
2006.09.02        2
1920.00.00.b      2
2012.09.02.b      2
1990.05.10        2
1915.07.06.a.R    2
1975.07.15        1
2015.04.24.c      1
1894.06.15.a.R    1
1967.08.25        1
ND-0130           1
1995.07.28.d      1
1970.12.03        1
1993.06.11.a      1
1961.09.24.a      1
2006.09.04        1
1910.11.28        1
1953.07.11        1
2003.10.31.c      1
1911.10.26        1
                 ..
1980.11.18        1
2008.03.15        1
1950.01.16        1
1947.04.20        1
1929.12.26        1
1940.00.00.f      1
2011.12.11        1
1880.00.00.a      1
1957.12.18        1
1952.04.00        1
ND-0133           1
2010.12.01.a      1
1886.01.01        1
1960.11.11        1
1997.09.08        1
1935.04.12.R      1
1938.01.14        1
2015.09.01        1
1853.11.08        1


# les premiers points de cleaning à faire:
1.passer les données en dataframe -->OK<br>
2.inspecter les données-->OK<br>
3.renommer les colonnes pour une meilleures lecture-->OK<br>
4.1 renomer la colonne "Sex " en supprimant l'espace --> OK<br>
5.donner le bon type aux données (int, float, DATE, string, boolean)<br>
6.on peut fusionner dans la colonne Type les données "boat" et "boating" -->OK<br>
7.dans la colonne Country : mettre tous les pays en Majuscule, corriger le nom "ST. MAARTIN" en "ST. MARTIN",<br> 
   supprimer la ligne "Between PORTUGAL & INDIA" et trouver d'autres info du genre--> OK<br>
8.on peut supprimer les carctères spéciaux (les accents)<br>
9.pour la colonne 'Sex', regarger pourquoi il y a 2 M, pour les 3 autres on les remplace par du non défini. --> OK<br>
10.pour les ages, on peut créér une colonne qui sont des tranches d'age-->OK<br>
11.pour 'Fatal', nettoyer les différents 'N' ajouter ce qui n'est pas connus par 'UNKNOWN' et 'F' par 'Y'-->OK<br>
12.supprimer une des colonnes "href" car doublon. Si espace dans l'url ajouter un '-' -->OK<br>
13.supprimer les doublons en dernier quand le fichier sera la plus clean à mon goût<br>

# Travail sur la colonne 'Sex'

In [20]:
df['SEX'].value_counts() 

M          4837
F           585
UNKNOWN       3
Name: SEX, dtype: int64

pour la colonne 'Sex', je corrige pour conserver 3 infos : M, F ou UNKNOWN<br/>
Remarque = possibilité de faire un dico pour y mettre mes modif désirées

In [21]:
df['SEX'] =df['SEX'].replace(to_replace =["lli", "N", "."], value ="UNKNOWN") 
df['SEX'] =df['SEX'].replace(to_replace =["M "], value ="M")

"""
--- Autre solution ---
df.loc[df['SEX'] == 'lli']
df.at[1400,'SEX']='UNKNOWN'
df.loc[df['SEX'] == 'N']
df.at[4708,'SEX']='UNKNOWN'
df.loc[df['SEX'] == '.']
df.at[5201,'SEX']='UNKNOWN'
df.loc[df['SEX'] == 'M ']
df.at[341,'SEX']='M'
df.at[1363,'SEX']='M
'"""
#pour Vérification ==> df.loc[df['Sex'] == 'UNKNOWN']
df['SEX'].value_counts() 


M          4837
F           585
UNKNOWN       3
Name: SEX, dtype: int64

# Travail sur la colonne 'FATAL (Y/N)'

In [22]:
df['FATAL (Y/N)'].value_counts() 

N          4315
Y          1552
UNKNOWN      94
 N            8
F             1
#VALUE!       1
n             1
N             1
Name: FATAL (Y/N), dtype: int64

Je choisis de ne garder que 3 infos pour la colonne 'FATAL (Y/N)'<br/>
pour cela je nettois les elements plus haut

In [23]:
df['FATAL (Y/N)'] =df['FATAL (Y/N)'].replace(to_replace =[" N", "n", "N "], value ="N") 
df['FATAL (Y/N)'] =df['FATAL (Y/N)'].replace(to_replace =["F"], value ="Y")
df['FATAL (Y/N)'] =df['FATAL (Y/N)'].replace(to_replace =["#VALUE!"], value ="UNKNOWN")
df['FATAL (Y/N)'].value_counts() 

N          4325
Y          1553
UNKNOWN      95
Name: FATAL (Y/N), dtype: int64

# Travail sur les colonnes 'YEAR' et 'DATE'
Je commence par regarder coment se présentent les 2 colonnes avec value_counts()

In [24]:
df['YEAR'].value_counts()

2015    139
2011    128
2014    125
0       124
2013    122
2008    121
2009    120
2012    117
2007    112
2006    103
2016    103
2005    103
2010    101
2000     97
1959     93
1960     93
2001     92
2004     92
2003     92
2002     88
1962     86
1961     78
1995     76
1964     66
1998     65
1999     65
1996     61
1963     61
1966     58
1997     57
       ... 
1785      1
1834      1
1791      1
1733      1
1721      1
1637      1
1617      1
77        1
5         1
1703      1
1755      1
1767      1
1771      1
1779      1
1787      1
1803      1
1749      1
1807      1
1811      1
1819      1
1805      1
1831      1
1555      1
1738      1
1859      1
1742      1
1758      1
1818      1
1822      1
1595      1
Name: YEAR, Length: 232, dtype: int64

il y a 124 annees avec 0 pour infos et 1 champs avec la valeur 5 et 1 autre avec la valeur 77<br/>

In [25]:
df['DATE'].value_counts()

1957                    11
1942                     9
1956                     8
1958                     7
1941                     7
1950                     7
No date                  6
1949                     6
1940                     5
1955                     5
No date, Before 1963     5
1954                     5
Aug-56                   5
1959                     5
Oct-60                   5
28-Jul-95                5
05-Oct-03                5
12-Apr-01                5
1970s                    5
1890                     4
09-Jan-10                4
28-Dec-14                4
1898                     4
1961                     4
1960                     4
Reported 10-Oct-1906     4
1952                     4
14-Jun-12                4
23-Jan-70                4
1938                     4
                        ..
Reported 05-May-1917     1
20-Jul-96                1
1950.07.19               1
Reported 21-Mar-2013     1
25-Dec-68                1
20-Apr-74                1
2

Dans la colonne 'DATE', je constate qu'il y a des cellules qui n'ont pas le format DATE et qui commencent par un "string". 
Je fais 2 travaux en parallèle : <br/>
1) Je cherche les lignes avec les champs 0 dans la colonne 'YEAR' et pour les renseigner (si nécessaire) en me servant de la colonne DATE<br/>
2) je choisis de remplacer ce qui n'est pas en format DATE par l'info "No DATE"<br/>
Pour cela, je crée une df temporaire où je split en 3 colonnes les infos de la colonne 'DATE' et je renomme les colonnes en 'jour', 'mois','Annee'<br/>
Je concatene ce nouveau tableau avec la colonne 'YEAR' de la df d'origine<br/>

In [26]:
df_DATE= df['DATE'].str.split('\W',n=3,expand =True)
df_DATE.rename(columns={0:'Jour', 1:'Mois', 2:'Annee', 3:'Autres'},inplace=True)

result = pd.concat([df_DATE, df['YEAR']], axis=1, join='inner')
result.head()

,Jour,Mois,Annee,Autres,YEAR
0,18,Sep,16,None,2016
1,18,Sep,16,None,2016
2,18,Sep,16,None,2016
3,17,Sep,16,None,2016
4,16,Sep,16,None,2016


J'identifie les lignes où YEAR = 0 et où Jour n'est pas un nombre

In [27]:
result_list = result.loc[((result['YEAR']==0) & (result['Jour'].str.contains('^\D')))]
result_list

,Jour,Mois,Annee,Autres,YEAR
5868,Ca,,214,B.C.,0
5869,Ca,,336,B.C..,0
5871,Ca,,725,B.C.,0
5873,Before,2016,None,None,0
5874,Before,Oct,2009,None,0
5875,Before,1934,None,None,0
5876,Before,1934,None,None,0
5878,Before,1930,None,None,0
5880,Before,1909,None,None,0
5881,Before,2012,None,None,0


In [28]:
result_list['Jour'].value_counts()

Before     62
No         26
Ca          3
World       2
Between     2
Said        1
Early       1
A           1
Beforer     1
Circa       1
Name: Jour, dtype: int64

Je créé une liste des index pour ensuite remplacer les données qui ne sont pas des DATEs 

In [32]:
rl = result_list['Jour'].index.to_list()

#rl

Je choisi d'ajouter une colonne nommée 'DATE_CLEAN', pour conserver les DATEs nettoyées.<br/>
    Si j'ai une DATE je conserve la DATE<br/>
    Sinon je mets "No DATE" pour la liste d'index identifiés plus haut

In [33]:
df = df.assign(DATE_CLEAN="")
df['DATE_CLEAN']=df['DATE']

In [34]:
for r in rl:
    df.at[r,'DATE_CLEAN']='No DATE'

In [35]:
df['DATE_CLEAN'].value_counts()

No DATE                 100
1957                     11
1942                      9
1956                      8
1941                      7
1958                      7
1950                      7
1949                      6
1954                      5
Aug-56                    5
Oct-60                    5
1970s                     5
12-Apr-01                 5
1940                      5
1959                      5
28-Jul-95                 5
1955                      5
05-Oct-03                 5
1952                      4
1995                      4
28-Dec-14                 4
1938                      4
23-Jan-70                 4
27-Dec-08                 4
09-Jul-94                 4
14-Jun-12                 4
09-Jan-10                 4
Reported 10-Oct-1906      4
1898                      4
27-Jul-52                 4
                       ... 
20-Jul-96                 1
1950.07.19                1
Reported 21-Mar-2013      1
20-Apr-74                 1
15-Jul-11           

Dans la df temporaire 'result', j'ai aussi vu des info sous le format 1845-1853<br/>
Pour ces formats, je change aussi les cellules de la colonne 'DATE_CLEAN' par "No DATE"

In [36]:
result_list1 = result.loc[((result['YEAR']==0) & ((result['Jour'].str.contains('^\d{4}')) & (result['Mois'].str.contains('^\d{4}'))))]
result_list1

,Jour,Mois,Annee,Autres,YEAR
5879,1880,1899,None,None,0
5893,1896,1913,None,None,0
5949,1941,1945,None,None,0
5953,1941,1945,None,None,0
5954,1941,1945,None,None,0
5955,1941,1942,None,None,0
5959,1940,1946,None,None,0
5989,1900,1905,None,None,0
5990,1883,1889,None,None,0
5991,1845,1853,None,None,0


In [37]:
rl1=result_list1['Jour'].index.to_list()
for r1 in rl1:
    df.at[r1,'DATE_CLEAN']='No DATE'
df['DATE_CLEAN'].value_counts()

No DATE                 110
1957                     11
1942                      9
1956                      8
1958                      7
1950                      7
1941                      7
1949                      6
05-Oct-03                 5
1959                      5
Aug-56                    5
1954                      5
1955                      5
1970s                     5
Oct-60                    5
1940                      5
28-Jul-95                 5
12-Apr-01                 5
1995                      4
09-Jul-94                 4
1898                      4
1960                      4
1952                      4
1945                      4
14-Jun-12                 4
1876                      4
28-Dec-14                 4
1890                      4
Reported 10-Oct-1906      4
20-Sep-15                 4
                       ... 
Reported 05-May-1917      1
20-Jul-96                 1
1950.07.19                1
25-Dec-68                 1
30-Nov-12           

Je regarde maintenant s'il y a d'autres modifications à réaliser

In [38]:
result_list2 = result.loc[((result['YEAR']==0) & (result['Jour'].str.contains('\w')==False) & (result['Mois'].str.contains('\D')))]
result_list2

,Jour,Mois,Annee,Autres,YEAR
5950,,During,the,"war"" 1943-1945",0
5951,,Before,the,"war""",0


In [39]:
#rl2=[5950,5951,5865]
rl2=result_list2['Jour'].index.to_list()
for r2 in rl2:
    df.at[r2,'DATE_CLEAN']='No DATE'
df['DATE_CLEAN'].value_counts()

No DATE                 112
1957                     11
1942                      9
1956                      8
1950                      7
1941                      7
1958                      7
1949                      6
Oct-60                    5
Aug-56                    5
05-Oct-03                 5
1955                      5
12-Apr-01                 5
1959                      5
1954                      5
1940                      5
1970s                     5
28-Jul-95                 5
1960s                     4
28-Dec-14                 4
Reported 10-Oct-1906      4
1945                      4
27-Jul-52                 4
1938                      4
1890                      4
1960                      4
09-Jul-94                 4
23-Jan-70                 4
14-Jun-12                 4
1898                      4
                       ... 
Reported 05-May-1917      1
20-Jul-96                 1
1950.07.19                1
25-Dec-68                 1
30-Nov-12           

Pour toutes ces modifications, je n'ai pas eu besoin de toucher à la colonne 'YEAR'<br/>
Je prends maintenant les cas où la colonne 'YEAR' est renseignée 

In [40]:
result_list3 = result.loc[(result['YEAR']!=0) & (result['Jour'].str.contains('Reported')==True) & (result['Autres'].str.contains('\d')==True)]
result_list3.head()

,Jour,Mois,Annee,Autres,YEAR
33,Reported,,14,Jul-2016,2016
34,Reported,08,Jul,2016,2016
85,Reported,03,Mar,2016,2016
90,Reported,10,Feb,2016,2016
100,Reported,11,Jan,2016,2016


Je supprime le mot 'Reported' des cellules pour ne conserver que la DATE

In [41]:
df['DATE_CLEAN'] = df['DATE_CLEAN'].str.lstrip('Reported')
df['DATE_CLEAN']

0        18-Sep-16
1        18-Sep-16
2        18-Sep-16
3        17-Sep-16
4        16-Sep-16
5        15-Sep-16
6        11-Sep-16
7        07-Sep-16
8        06-Sep-16
9        05-Sep-16
10       05-Sep-16
11       04-Sep-16
12       01-Sep-16
13       29-Aug-16
14       29-Aug-16
15       27-Aug-16
16       25-Aug-16
17       07-Aug-16
18       06-Aug-16
19       04-Aug-16
20       29-Jul-16
21       28-Jul-16
22       28-Jul-16
23       27-Jul-16
24       26-Jul-16
25       24-Jul-16
26       23-Jul-16
27       23-Jul-16
28       20-Jul-16
29       17-Jul-16
           ...    
5962       No DATE
5963       No DATE
5964       No DATE
5965       No DATE
5966       No DATE
5967       No DATE
5968       No DATE
5969       No DATE
5970       No DATE
5971       No DATE
5972    1920 -1923
5973       No DATE
5974       No DATE
5975       No DATE
5976       No DATE
5977       No DATE
5978       No DATE
5979       No DATE
5980       No DATE
5981       No DATE
5982       No DATE
5983       N

In [42]:
df['DATE_CLEAN'] = df['DATE_CLEAN'].str.lstrip()
[df['DATE_CLEAN'].loc[df['DATE_CLEAN'].str.contains('happened')]]

[1431    19-Jul-2004 Reported to have happened  "on the...
 5535        14-Sep-1883 (probably happened Ca. 1843/1844)
 Name: DATE_CLEAN, dtype: object]

En faisant un value_counts sur la colonne DATE_CLEAN, j'ai repéré quelques points à modifier<br/>
#rl2 mettre no DATE =[5950,5951,5865,5866,5867,4088,5341,5357,5670,1868]
#mettre 'YEAR' à 0 et 'DATE_CLEAN' à No DATE=[5865,5866,5867]
#retirer le point et la lettre à la fin = [1133,1134,2990]
#réécrire les DATEs pour = [5776,5794,3883]

In [43]:
result_list4 = result.loc[(result['YEAR']!=0) & (df['DATE'].str.contains('happened')==True) & (result['Autres'].str.contains('\d')==True)]
result_list4

,Jour,Mois,Annee,Autres,YEAR
1431,,19,Jul,"2004 Reported to have happened ""on the weekend""",2004
5535,Reported,14,Sep,1883 (probably happened Ca. 1843/1844),1883


In [44]:
df.at[[5950,5951,5865,5866,5867,4088,5341,5357,5670,1868,5855,3184,3801,5323,5626,5662,5972],'DATE_CLEAN']='No DATE'
df.at[[5865,5866,5867,5535],'DATE_CLEAN']='No DATE'
df.at[[5865,5866,5867],'YEAR']=0
df.at[[1133,1134],'YEAR']='2007'
df.at[[1133,1134],'DATE_CLEAN']='19-Jul-2007'
df.at[[2990],'YEAR']='1975'
df.at[[2990],'DATE_CLEAN']='26-Jul-1975'
df.at[[5776],'DATE_CLEAN']='16-Jul-1844'
df.at[[5794],'DATE_CLEAN']='26-Jul-1836'
df.at[[3883],'DATE_CLEAN']='02-Feb-1959'
df.at[1431,'DATE_CLEAN']='19 Jul 2004'

In [45]:
 df['DATE_CLEAN'].value_counts()

No DATE          128
1957              11
1942               9
1956               8
1958               7
1950               7
1941               7
1949               6
1898               5
1938               5
05-Oct-03          5
1959               5
1940               5
1955               5
12-Apr-01          5
1954               5
1970s              5
Oct-60             5
28-Jul-95          5
Aug-56             5
23-Jan-70          4
27-Dec-08          4
1960               4
09-Jul-94          4
20-Sep-15          4
1961               4
10-Oct-1906        4
28-Dec-14          4
1929               4
1995               4
                ... 
25-Dec-68          1
20-Apr-74          1
25-Jul-49          1
26-Jun-1867        1
20-Mar-67          1
26-Nov-1909        1
11-Aug-07          1
14-Jan-07          1
16-Nov-1895        1
01-Jun-11          1
2004               1
08-Jan-89          1
15-Jul-11          1
14-Mar-66          1
24-Mar-00          1
Before 1878        1
23-Mar-70    

Je décide de mettre 'No DATE' pour les DATEs au format 1xxxs

In [46]:
df['DATE_CLEAN'] = df['DATE_CLEAN'].str.lstrip()
df['DATE_CLEAN'].loc[df['DATE'].str.contains('\d\d\d\ds')].value_counts()

No DATE    8
1970s      5
1960s      4
1700s      3
1980s      2
1950s      2
1920s      1
1820s      1
Name: DATE_CLEAN, dtype: int64

In [47]:
df.at[[5852,5853,5854,5816,5028,2855,2857,3183,3187,3188,3189,3190,3802,5931,5932,5933,4264,5904],'DATE_CLEAN']='No DATE'

In [48]:
"""
    #Ancienne méthode
    
pat=[r'Reported\s\d{2}-\w{3}-\d{4}',r'Reported  16-Nov-2005',r'Reported      23-Aug-1998',r'Before\s\w{3}\s\d{4}',r'Before\s\d{4}',r'Circa\s\d{4}',r'Between?',
     r'No DATE, No DATE',r'No DATE No DATE',r'No DATE\W\w*',r'No DATE\W\d*',r'No DATE\s\w*',r'No DATE\s\W\w*',
     r'No DATE\s\W\d*',r'Early\s\w+',r'No DATE\s\w+',r'No DATE and\s\w+',r'\d{4}-\d{4}|\d{4}\s-\d{4}',r'\d{4}s','World War II',
     'Ca.\s\d+\sB.C.','Ca.\s\d+.B.C\W+','\d{3}\sB.C.']
for x in pat:
    df['DATE_CLEAN'] =df['DATE_CLEAN'].replace(regex =x, value ="No DATE")

#df['DATE_CLEAN'][df['DATE_CLEAN'].value_counts()].head()
df[df['DATE_CLEAN'].str.match('No DATE')==True]"""

df['DATE_CLEAN'].value_counts()

No DATE          146
1957              11
1942               9
1956               8
1950               7
1958               7
1941               7
1949               6
Oct-60             5
1898               5
12-Apr-01          5
1955               5
28-Jul-95          5
1940               5
05-Oct-03          5
1959               5
Aug-56             5
1938               5
1954               5
23-Jan-70          4
1929               4
1995               4
1945               4
27-Dec-08          4
09-Jan-10          4
1961               4
1952               4
20-Sep-15          4
28-Dec-14          4
10-Oct-1906        4
                ... 
25-Dec-68          1
20-Apr-74          1
25-Jul-49          1
26-Jun-1867        1
20-Mar-67          1
26-Nov-1909        1
11-Aug-07          1
14-Jan-07          1
16-Nov-1895        1
01-Jun-11          1
2004               1
08-Jan-89          1
15-Jul-11          1
14-Mar-66          1
24-Mar-00          1
Before 1878        1
23-Mar-70    

In [49]:
result_list5 = result.loc[(result['YEAR']!=0) & (df['DATE_CLEAN'].str.contains('^\w{3}-')) ]
result_list5.head()

,Jour,Mois,Annee,Autres,YEAR
151,Sep,15,None,None,2015
282,Sep,14,None,None,2014
303,Aug,14,None,None,2014
496,Dec,12,None,None,2012
579,Apr,13,None,None,2012


In [50]:
rl5=result_list5['Jour'].index.to_list()
for r5 in rl5:
    df.at[r5,'DATE_CLEAN']='No DATE'
df['DATE_CLEAN'].value_counts()

No DATE        465
1957            11
1942             9
1956             8
1958             7
1941             7
1950             7
1949             6
1938             5
05-Oct-03        5
28-Jul-95        5
1940             5
1898             5
1959             5
1954             5
1955             5
12-Apr-01        5
1890             4
1876             4
1945             4
1995             4
1960             4
14-Jun-12        4
28-Dec-14        4
09-Jul-94        4
23-Jan-70        4
27-Dec-08        4
27-Jul-52        4
1952             4
09-Jan-10        4
              ... 
08-Feb-24        1
15-Jul-16        1
23-Sep-11        1
06-Nov-08        1
26-Apr-07        1
2004             1
01-Jun-11        1
16-Nov-1895      1
14-Jan-07        1
11-Aug-07        1
26-Nov-1909      1
26-Jun-1867      1
29-Nov-13        1
20-Apr-74        1
06-Oct-88        1
25-Dec-68        1
1950.07.19       1
20-Jul-96        1
12-Feb-11        1
08-Jan-89        1
20-Mar-67        1
15-Jul-11   

In [51]:
result_list6 = result.loc[(result['YEAR']!=0) & (df['DATE_CLEAN'].str.contains('^\d{4}')) ]
result_list6.head()

,Jour,Mois,Annee,Autres,YEAR
366,2014,None,None,None,2014
1485,2004,None,None,None,2004
1854,2000,None,None,None,2000
1918,1999,None,None,None,1999
1919,1999,None,None,None,1999


In [52]:
rl6=result_list6['Jour'].index.to_list()
for r6 in rl6:
    df.at[r6,'DATE_CLEAN']='No DATE'
df['DATE_CLEAN'].value_counts()

No DATE        768
28-Jul-95        5
12-Apr-01        5
05-Oct-03        5
10-Oct-1906      4
27-Jul-52        4
27-Dec-08        4
09-Jul-94        4
23-Jan-70        4
14-Jun-12        4
28-Dec-14        4
09-Jan-10        4
20-Sep-15        4
11-Aug-97        3
07-Jan-74        3
22-May-04        3
19-Aug-07        3
31-May-97        3
29-Aug-82        3
27-Aug-14        3
06-Jan-61        3
13-Mar-77        3
26-Jun-12        3
31-Oct-03        3
08-Mar-92        3
11-Jun-62        3
25-Jun-13        3
01-Apr-34        3
24-Jul-09        3
08-Aug-1899      3
              ... 
17-Jan-98        1
29-Nov-1889      1
Circa 1872       1
01-Jul-53        1
28-Jul-54        1
14-Jul-1907      1
09-Jun-1873      1
24-Mar-62        1
21-Mar-1938      1
08-Feb-24        1
15-Jul-16        1
23-Sep-11        1
06-Nov-08        1
26-Apr-07        1
01-Jun-11        1
16-Nov-1895      1
14-Jan-07        1
11-Aug-07        1
26-Nov-1909      1
26-Jun-1867      1
29-Nov-13        1
25-Jul-49   

J'applique les dernières modifications

In [53]:
df['DATE_CLEAN'].loc[(df['DATE_CLEAN'].str.contains('^\d{4}')) | (df['DATE'].str.contains('\w or \w')) | (df['DATE'].str.contains('^19\d+\s-\s\d+'))]

2628             Mid Jul-1985 or mid Jul-1986
3300                        13 or 30-May-1967
4261                                  No DATE
4290                                  No DATE
4937               07-Mar-1925 or 27-Mar-1925
5739                          Sep or Oct-1853
5741                                  No DATE
5750    "Anniversary Day" 22-Jan-1850 or 1852
5872                             1990 or 1991
5877                                    2009?
5906                                    2003?
5956                              1940 - 1950
5957                              1940 - 1950
5958                              1940 - 1950
Name: DATE_CLEAN, dtype: object

In [54]:
df.at[[2628,3300,4261,4290,4937,5739,5741,5750,5872,5877,5906,5956,5957,5958],'DATE_CLEAN']='No DATE'
df['DATE_CLEAN'].value_counts()

No DATE        779
05-Oct-03        5
28-Jul-95        5
12-Apr-01        5
10-Oct-1906      4
09-Jul-94        4
27-Dec-08        4
28-Dec-14        4
14-Jun-12        4
09-Jan-10        4
27-Jul-52        4
23-Jan-70        4
20-Sep-15        4
24-Jul-09        3
28-Dec-61        3
16-May-98        3
08-Aug-1899      3
22-May-04        3
06-Mar-09        3
06-Jan-61        3
20-Apr-03        3
19-Aug-07        3
20-Aug-06        3
18-Mar-14        3
07-Jan-74        3
27-Sep-02        3
30-Aug-08        3
28-Mar-08        3
19-Aug-01        3
07-Jul-12        3
              ... 
17-Jan-98        1
29-Nov-1889      1
Circa 1872       1
01-Jul-53        1
28-Jul-54        1
14-Jul-1907      1
09-Jun-1873      1
24-Mar-62        1
21-Mar-1938      1
08-Feb-24        1
15-Jul-16        1
23-Sep-11        1
06-Nov-08        1
26-Apr-07        1
01-Jun-11        1
16-Nov-1895      1
14-Jan-07        1
11-Aug-07        1
26-Nov-1909      1
26-Jun-1867      1
29-Nov-13        1
25-Jul-49   

In [55]:
"""result_list7 = (df.loc[df['DATE_CLEAN']=='No DATE']).index.to_list()

for r7 in result_list7:
    df.at[r7,'DATE_CLEAN']='0'
df['DATE_CLEAN']"""



"result_list7 = (df.loc[df['DATE_CLEAN']=='No DATE']).index.to_list()\n\nfor r7 in result_list7:\n    df.at[r7,'DATE_CLEAN']='0'\ndf['DATE_CLEAN']"

In [56]:
"""from DATEtime import DATEtime

# Complete the call to convert the DATE column
df['DATE_CLEAN'].value_counts()
df['DATE_CLEAN'] = pd.to_DATEtime(df['DATE_CLEAN'],errors='ignore')
df.info()"""


"from DATEtime import DATEtime\n\n# Complete the call to convert the DATE column\ndf['DATE_CLEAN'].value_counts()\ndf['DATE_CLEAN'] = pd.to_DATEtime(df['DATE_CLEAN'],errors='ignore')\ndf.info()"

# Travail sur la colonne "Type"

In [57]:
df[df['DATE'].str.match(r'No DATE')]

,CASE NUMBER,DATE,YEAR,TYPE,COUNTRY,AREA,LOCATION,ACTIVITY,NAME,SEX,...,PDF,HREF FORMULA,HREF,CASE NUMBER.1,CASE NUMBER.2,ORIGINAL ORDER,UNNAMED: 22,UNNAMED: 23,DATE_clean,DATE_CLEAN


In [58]:
df['TYPE'].value_counts()

Unprovoked      4386
Provoked         557
Invalid          519
Sea Disaster     220
Boat             200
Boating          110
Name: TYPE, dtype: int64

pour la colonne 'TYPE', je fusionne Boat et Boating

In [59]:
df['TYPE'] =df['TYPE'].replace(to_replace =["Boating"], value ="Boat")
df['TYPE'].value_counts()

Unprovoked      4386
Provoked         557
Invalid          519
Boat             310
Sea Disaster     220
Name: TYPE, dtype: int64

# Travail sur la colonne 'COUNTRY'
    #mettre tous les pays en Majuscule,<br/> 
    corriger le nom "ST. MAARTIN" en "ST. MARTIN",<br/>
    #remplacer les NaN par "NC"<br/>
    #passer en "NC" les lignes "Between PORTUGAL & INDIA" ou avec un "?" ou avec "/"<br/>
    #supprimer les espaces avant un mot<br/>

In [60]:
(df['COUNTRY'].value_counts()).head(20)

USA                 2116
AUSTRALIA           1279
SOUTH AFRICA         565
PAPUA NEW GUINEA     133
NEW ZEALAND          125
BRAZIL               102
BAHAMAS               98
MEXICO                81
ITALY                 71
FIJI                  62
PHILIPPINES           59
REUNION               57
NEW CALEDONIA         51
MOZAMBIQUE            44
CUBA                  42
SPAIN                 40
INDIA                 37
EGYPT                 36
CROATIA               34
JAPAN                 32
Name: COUNTRY, dtype: int64

In [61]:
#passer toutes les données en majuscules
df['COUNTRY'] = df['COUNTRY'].str.upper()
#identification des NaN et remplacement des NaN par NC (Non Communiqué) car il est difficile après de faire des 'replace'
    #df['COUNTRY'].isna()
    #df.loc[df['COUNTRY'].isna()]
df['COUNTRY']=df['COUNTRY'].fillna(value='NC')

df['COUNTRY'] =df['COUNTRY'].replace(to_replace =["ST. MAARTIN"], value ="ST. MARTIN")

#supprimer le ? de trop
df['COUNTRY'] = df['COUNTRY'].str.replace('[?]','')
    #df['COUNTRY']
    #df[df['COUNTRY'].str.match('NC')].head()

#remplacer les emplacements incertains par "NC" (qui ont "beetween", "/" dans l'intitulé)
    #df.loc[df['COUNTRY'] == 'NC']
pat5 = df.loc[df['COUNTRY'].str.contains(r'( / )')]
df['COUNTRY'] =df['COUNTRY'].replace(regex =[pat5], value ="NC") 

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [62]:
df['COUNTRY'].str.lstrip()
#supprimer l'espace au début 
#pat5 = df.loc[df['COUNTRY'].str.contains(r'(^/s)')]

df.columns = df.columns.str.lstrip()
(df['COUNTRY'].value_counts()).head(20)

USA                 2116
AUSTRALIA           1279
SOUTH AFRICA         565
PAPUA NEW GUINEA     133
NEW ZEALAND          125
BRAZIL               102
BAHAMAS               98
MEXICO                81
ITALY                 71
FIJI                  65
PHILIPPINES           59
REUNION               57
NEW CALEDONIA         51
NC                    50
MOZAMBIQUE            44
CUBA                  42
SPAIN                 40
INDIA                 37
EGYPT                 36
CROATIA               34
Name: COUNTRY, dtype: int64

In [63]:
pat6 = df.loc[df['COUNTRY'].str.contains(r'(^ )')]
#pat6
df['COUNTRY'] =df['COUNTRY'].replace(regex =[pat6], value ="")

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


# Vérificaton des infos dans les colonnes 'UNNAMED: 22' & 'UNNAMED: 23'

In [65]:
df['UNNAMED: 22'].value_counts()

stopped here    1
Name: UNNAMED: 22, dtype: int64

In [66]:
df['UNNAMED: 23'].value_counts()

change filename    1
Teramo             1
Name: UNNAMED: 23, dtype: int64

Comme il n'y a pas d'infos nécessaires, je les supprime

In [67]:
df=df.drop(columns=['UNNAMED: 22','UNNAMED: 23'],axis=1)

# Travail sur la colonne "pdf"
    Je remplace 'pf' par 'PDF'

In [68]:
pat7 = df.loc[df['PDF'].str.contains(r'pf+')]
df['PDF'] =df['PDF'].replace(regex =[pat7], value ="pdf")
df.loc[df['PDF'].str.contains(r'pdf+')]

,CASE NUMBER,DATE,YEAR,TYPE,COUNTRY,AREA,LOCATION,ACTIVITY,NAME,SEX,...,SPECIES,INVESTIGATOR OR SOURCE,PDF,HREF FORMULA,HREF,CASE NUMBER.1,CASE NUMBER.2,ORIGINAL ORDER,DATE_clean,DATE_CLEAN
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,18-Sep-16,18-Sep-16
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,18-Sep-16,18-Sep-16
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,18-Sep-16,18-Sep-16
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,17-Sep-16,17-Sep-16
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,16-Sep-16,16-Sep-16
5,2016.09.15.R,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,15-Sep-16,15-Sep-16
6,2016.09.11,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,11-Sep-16,11-Sep-16
7,2016.09.07,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,07-Sep-16,07-Sep-16
8,2016.09.06,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,06-Sep-16,06-Sep-16
9,2016.09.05.b,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,05-Sep-16,05-Sep-16


# Travail sur la colonne "href"
Identification des NaN et remplacement des NaN par NC (Non Communiqué) car il est difficile après de faire des 'replace'

In [69]:
df['HREF'].isna()
df.loc[df['HREF'].isna()]
df['HREF']=df['HREF'].fillna(value='NC')
    #remplacer 'pf' par 'PDF'
pat8 = df.loc[df['HREF'].str.contains(r'pf+')]
df['HREF'] =df['HREF'].replace(regex =[pat8], value ="pdf")
df.loc[df['HREF'].str.contains(r'pdf+')]

,CASE NUMBER,DATE,YEAR,TYPE,COUNTRY,AREA,LOCATION,ACTIVITY,NAME,SEX,...,SPECIES,INVESTIGATOR OR SOURCE,PDF,HREF FORMULA,HREF,CASE NUMBER.1,CASE NUMBER.2,ORIGINAL ORDER,DATE_clean,DATE_CLEAN
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,18-Sep-16,18-Sep-16
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,18-Sep-16,18-Sep-16
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,18-Sep-16,18-Sep-16
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,17-Sep-16,17-Sep-16
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,16-Sep-16,16-Sep-16
5,2016.09.15.R,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,15-Sep-16,15-Sep-16
6,2016.09.11,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,11-Sep-16,11-Sep-16
7,2016.09.07,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,07-Sep-16,07-Sep-16
8,2016.09.06,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,06-Sep-16,06-Sep-16
9,2016.09.05.b,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,05-Sep-16,05-Sep-16


# Suppression de la colonne "href formula" 
car identique à la colonne "href"

In [70]:
df = df.drop(columns=['HREF FORMULA'],axis=1)

# Travail sur la colonne "Time"
    remplacer les NaN

In [71]:
df['TIME'].value_counts()
df.loc[df['TIME'].isna()]
df['TIME']=df['TIME'].fillna(value='NC')

Je change la syntaxe des élements ayant une heure correcte mais mal écrite pour la suite du nettoyage.<br/>
En effet, avec ce format de DATE je vais pouvoir remplir la colonne "AM_PM" créée et ensuite les changer en AM, PM ou NC.
        #A faire plus tard  --> df['TIME']= df['TIME'].str.replace('h',':',regex=True)

In [72]:
df['TIME'] =df['TIME'].replace(to_replace =["06j00"], value ="06h00")
df['TIME'] =df['TIME'].replace(to_replace =["8:04 PM"], value ="20h04")
df['TIME'] =df['TIME'].replace(to_replace =["15j45"], value ="15h45")
df['TIME'] =df['TIME'].replace(to_replace =["1300"], value ="13h00")
df['TIME'] =df['TIME'].replace(to_replace =["1600"], value ="16h00")
df['TIME'] =df['TIME'].replace(to_replace =["1500"], value ="15h00")
df['TIME'] =df['TIME'].replace(to_replace =["13h345"], value ="13h34")

### Creation d'un nouvelle colonne "AM_PM" pour y reseigner les infos suivantes : le matin-AM de l'apres-midi-PM si pas possible ou vide alors NC


In [73]:
df = df.assign(AM_PM="")
df['AM_PM']=df['TIME']

df[['TIME','AM_PM']].head(15)

,TIME,AM_PM
0,13h00,13h00
1,11h00,11h00
2,10h43,10h43
3,NC,NC
4,NC,NC
5,NC,NC
6,15h15,15h15
7,14h30,14h30
8,15h40,15h40
9,Late afternoon,Late afternoon


In [74]:
df['AM_PM'].sort_values(ascending=False)

1631                                        
3588                                   night
5645                                    dusk
2485                                       X
1149                                  Sunset
1664                                  Sunset
1593                                  Sunset
1214                                  Sunset
250           Sometime between 06h00 & 08hoo
1718                    Shortly before 13h00
1619                  Shortly after midnight
4488                Ship aban-doned at 03h10
2767                          Prior to 10h37
1801    Possibly same incident as 2000.08.21
1952                                    P.M.
3752                                    P.M.
3795                                    P.M.
4332                                    P.M.
3923                                    P.M.
4005                                    P.M.
4188                                    P.M.
1842                                    P.M.
1673      

Retour au travail sur la colonne "AM_PM"<br/>
    Je remplace toutes les heures en info AM ou PM suivant le format

In [75]:
pat9=(df['AM_PM'].str.contains(r'[0-1][0-2]h[0-5][0-9]'))
pat10=(df['AM_PM'].str.contains(r'([1]?[2-9]|2[0-3])h[0-5][0-9]'))

df['AM_PM'] = df['TIME']
mask1 = (pat9==True)
df.loc[mask1, 'AM_PM'] = 'AM'


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [76]:
mask2 = (pat10==True)
df.loc[mask2, 'AM_PM'] = 'PM'
df['AM_PM'].value_counts()

NC                                                                       3213
PM                                                                       1822
AM                                                                        422
Afternoon                                                                 174
Morning                                                                   107
Night                                                                      58
Evening                                                                    33
Late afternoon                                                             32
Dusk                                                                       15
A.M.                                                                       12
P.M.                                                                       12
Early morning                                                              11
Midday                                                          

En checkant les valeurs uniques, on constate qu'il y a une info qui devrait renseigner la colonne "FATAL (Y/N)"<br/>
    Je cherche donc l'index 

In [78]:
df.loc[df['TIME'].str.contains('FATAL')]    
    #df.at[4776].value_counts()
    #je cherche ensuite si la colonne suivante n'a pas l'info
    #pour cela, j'ai réduit le tableau pour vérifier s'il n'y a pas de décalage 
df1 = df[['SEX','AGE','INJURY','FATAL (Y/N)','TIME','SPECIES','INVESTIGATOR OR SOURCE','PDF']]
df1.loc[4776]
#mais en fait c'est ok. Donc je ne corrige que la colonne "Fatal" pour cet index
df.at[4776,'FATAL (Y/N)']="Y"

In [79]:
pat11 = r'^\s$|^\s\s$|^\s\s\s$|Before daybreak|"shortly PM|After 04h00|Daytime|500|830|2 hours after Opperman|--|FATAL\s\s(\w+.)+|30 minutes after 1992.07.08.a|Possibly same incident as 2000.08.21|Sometime between 06h00 & 08hoo|>08h00|>06h45|<07h30|10h30 or 13h30|10h00 or 14h00|X'
pat12 = r'"Early evening"|Shortly after midnight|Nightfall|After midnight|A.M.|[A-Za-z]+\s[Aa]fterno*n|PM.|Midday.|"PM|"Night"|"After lunch"|Noon|[Aa]fterno*n|Lunchtime|Just before sundown|[Aa]fter\sno*n|Late night|[Mm]idnight|18h15-18h30|[Nn]ight+|[Ee]vening|evening+|"Evening"|[A-Za-z]+\s[Dd]usk|[Dd]usk|Dark|"After dark"|[Ss]unset|Midday|17h00 or 17h40|Just after 12h00|"shortly before dusk"' 
pat13 = r'P.M.|[A-Za-z]+\s[Mm]orning|Mid-morning|[Mm]orning|[Dd]awn|Just before dawn|[A-Za-z]+\s[Dd]awn|[Aa]fter\smidnight|Just before noon|"Just before 11h00"|Just AM|AM\s|Daybreak|2 hrs before PM|07h00 - 08h00|10h45-11h15|09h30 / 10h00|Prior to 10h37|11h00 / 11h30|09h00 - 09h30|09h00 -10h00|Before 07h00|10h00 -- 11h00|06h00 -- 07h00|Between 05h00 & 08h00|Between 06h00 & 07h20|10h00 / 11h00|03h45 / 04h00|08h00 / 09h30|Before 10h30|Between 11h00 & 12h00|After 04h00|11h01 -time of ship sinking|Ship aban-doned at 03h10' 
df['AM_PM'] =df['AM_PM'].replace(regex =[pat11], value ="NC") 
df['AM_PM'] =df['AM_PM'].replace(regex =[pat12], value ="PM")
df['AM_PM'] =df['AM_PM'].replace(regex =[pat13], value ="AM")
df.at[62,'AM_PM']='AM'

In [80]:
df[['TIME','AM_PM']].head(15)

,TIME,AM_PM
0,13h00,PM
1,11h00,AM
2,10h43,AM
3,NC,NC
4,NC,NC
5,NC,NC
6,15h15,PM
7,14h30,PM
8,15h40,PM
9,Late afternoon,PM


In [81]:
df['AM_PM'].value_counts()

NC                                                                       3231
PM                                                                       2196
AM                                                                        564
FATAL  (Wire netting installed at local beaches after this incident.)       1
Name: AM_PM, dtype: int64

In [82]:
df.loc[df['TIME'].str.contains(r'FATAL+')]

,CASE NUMBER,DATE,YEAR,TYPE,COUNTRY,AREA,LOCATION,ACTIVITY,NAME,SEX,...,SPECIES,INVESTIGATOR OR SOURCE,PDF,HREF,CASE NUMBER.1,CASE NUMBER.2,ORIGINAL ORDER,DATE_clean,DATE_CLEAN,AM_PM
4776,1931.08.31,31-Aug-31,1931,Unprovoked,CUBA,Havana Province,"Vedado Baths, Miramar, Havana",Swimming,Manuel Romero,M,...,NaN,"NYTimes, 9/2/1931",1931.08.31-Romero.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,1931.08.31,1931.08.31,1217,31-Aug-31,31-Aug-31,FATAL (Wire netting installed at local beache...


In [83]:
df.at[4776,'AM_PM']='AM'

In [84]:
df['AM_PM'].value_counts()

NC    3231
PM    2196
AM     565
Name: AM_PM, dtype: int64

In [85]:
df['TIME']= df['TIME'].str.replace('h',':',regex=True)
df['TIME']

0                13:00
1                11:00
2                10:43
3                   NC
4                   NC
5                   NC
6                15:15
7                14:30
8                15:40
9       Late afternoon
10      Late afternoon
11                  NC
12                  NC
13               15:00
14               14:00
15               17:00
16               16:00
17                  NC
18                  NC
19                  NC
20               11:30
21                  NC
22                  NC
23                  NC
24               12:00
25               19:05
26                  NC
27                  NC
28          After noon
29                  NC
             ...      
5962                NC
5963                NC
5964                NC
5965                NC
5966                NC
5967                NC
5968                NC
5969                NC
5970                NC
5971                NC
5972                NC
5973                NC
5974       

# Exportation d'un fichier csv propre

In [86]:
df.to_csv('shark_cleaning.csv', sep=',', index=False)